In [1]:
%load_ext Cython
%load_ext autoreload
%autoreload 2

import dask
dask.config.set(num_workers=8)

import satpy
satpy.config.set({'cache_dir': "D:/sat_data/cache/"})
satpy.config.set({'cache_sensor_angles': True})
satpy.config.set({'cache_lonlats': True})

from satpy import Scene
import dask.array as da
from glob import glob

from pyfires.PYF_detection import stage1_tests
from pyfires.PYF_basic import *

from pyspectral.rsr_reader import RelativeSpectralResponse
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [2]:
indir = 'D:/sat_data/HIM/'

ifiles_l15 = glob(indir + '*.DAT')

ifile_cld = f'{indir}/AHI-CMSK_v1r1_h09_s202308240540213_e202308240549407_c202308240557548.nc'

odir = 'D:/sat_data/SEV_FIRE/'


rad_dict = {'vi1': 1647.567,
            'vi2': 244.192,
            'swi': 10.259}

bdict = {'vi1_band': 'B03',
         'vi2_band': 'B05',
         'swi_band': 'B07',
         'lwi_band': 'B13',
         'lw2_band': 'B11'}

blist = []
for item in bdict:
    blist.append(bdict[item])

scn = initial_load(ifiles_l15, 'ahi_hsd', bdict, rad_dict, sw_bt_thresh=270, lw_bt_thresh=260)
#scn.save_datasets(base_dir=odir, enhance=False, dtype=np.float32)

vi_diff_arr = np.array(scn['VI1_DIFF'])
btd_arr = np.array(scn['BTD']) 
mir_arr = np.array(scn['SWI__BT'])
irr_arr = np.array(scn['SWI_RAD'])
lwi_arr = np.array(scn['LW1__BT'])
lwi_arr = np.array(scn['VI1_RAD'])
lsm = np.array(scn['LSM'])

cannot convert float NaN to integer


In [3]:
stg1_output = stage1_tests(scn['SWI__BT'].data,
                           scn['BTD'].data,
                           scn['VI1_DIFF'].data,
                           scn['SZA'].data,
                           scn['LSM'].data,
                           ksizes=[5, 7, 9])

scn['PFP'] = scn['SWI__BT'].copy()
scn['PFP'].attrs['name'] = 'PFP'
scn['PFP'].data = stg1_output
scn.save_dataset('PFP', base_dir=odir, enhance=False, dtype=np.float32)

In [4]:
%%cython

from libc.math cimport sqrt, isnan
cimport numpy as np
import numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_curwindow_pos(int xpos, int ypos, int winsize, int scn_width, int scn_height):
    """Find the correct position of the moving window in the input arrays.
    Inputs:
     - xpos: The x position of the candidate pixel
     - ypos: The y position of the candidate pixel
     - winsize: The radius of the window
     - scn_width: The width of the whole satellite scene
     - scn_height: The height of the whole satellite scene 
    Returns:
     - shape_sel: An array specifying the correct indices for the windowed region (x_0, x_1, y_0, y_1)
    """
    cdef int shape_sel[4]

    # Set up the output pixel selection
    shape_sel = [xpos - winsize, xpos + winsize + 1, ypos - winsize, ypos + winsize + 1]

    # Apply some range tests, so we can use cython in the non-bounds checking mode
    if shape_sel[0] < 0:
        shape_sel[0] = -999
    elif shape_sel[0] >= scn_width:
        shape_sel[0] = -999
    if shape_sel[1] < 0:
        shape_sel[0] = -999
    elif shape_sel[1] >= scn_height:
        shape_sel[0] = -999

    return shape_sel

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
@cython.initializedcheck(False)
cdef get_window_mea_stdv(int winsize,
                         float[:,:] cur_vid,
                         float cen_vid,
                         float[:,:] cur_btd,
                         float cen_btd,
                         float[:,:] cur_mir,
                         float cen_mir,
                         float[:,:] cur_irr,
                         float cen_irr,
                         float[:,:] cur_lwi,
                         float[:] btd_eqn,
                         float good_thresh_perc=0.65,
                         float max_btd=8,
                         float max_mir=330,
                         float min_mir=270):
    cdef int arrsize = winsize + winsize + 1
    cdef int npix = arrsize * arrsize
    cdef int i = 0
    cdef int j = 0

    cdef float n_good = 0
    cdef float perc_good = 0
    
    cdef float sum_val = 0
    cdef float mea_val = -999.
    cdef float std_val = -999.
    
    cdef float res[3]
    
    cdef float btd_slo = 0
    cdef float btd_int = 0
    cdef float bt_thresh = 0
    
    btd_slo = (btd_eqn[0] - btd_eqn[1]) / (btd_eqn[2] - btd_eqn[3])
    btd_int = btd_eqn[1]
 
    # Loop to find mean
    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            bt_thresh = btd_int + (cur_lwi[i,j] - btd_eqn[3]) * btd_slo
            if bt_thresh > btd_eqn[1]:
                bt_thresh = btd_eqn[1]
            if bt_thresh < btd_eqn[0]:
                bt_thresh = btd_eqn[0]
            
        
            if isnan(cur_vid[i, j]):
                continue
            if cur_vid[i,j] >= cen_vid:
                continue
            if cur_btd[i,j] >= cen_btd:
                continue
            if cur_btd[i,j] >= bt_thresh:
                continue
            if cur_mir[i,j] > 330:
                continue
            if cur_mir[i,j] < 270:
                continue
            sum_val = sum_val + cur_vid[i, j]
            n_good = n_good + 1
    perc_good = n_good / (npix - 9)
    
    # If we don't have enough good pixels, return with bad data flagged.
    if perc_good < good_thresh_perc:
        res[0] = perc_good
        res[1] = -999
        res[2] = -999
        return res
    
    mea_val = sum_val / n_good
    
     # Loop to find standard deviation
    sum_val = 0
    for i in range(0, arrsize):
        for j in range(0, arrsize):
            if i >= winsize - 1 and i <= winsize + 1 and j >= winsize - 1 and j <= winsize + 1:
                continue
            if isnan(cur_vid[i, j]):
                continue
            sum_val = sum_val + (cur_vid[i, j] - mea_val) * (cur_vid[i, j] - mea_val)
    std_val = sqrt(sum_val / n_good)
    
    res[0] = perc_good
    res[1] = mea_val
    res[2] = std_val
    return res 

    

def get_mea_std_window(float[:,:] vi_diff_data,
                       float[:,:] mir_bt_data,
                       float[:,:] btd_data,
                       float[:,:] irr_data,
                       float[:,:] lwi_data,
                       unsigned char[:,:] pfp,
                       int min_win=2,
                       int max_win=7,
                       float pix_win_percent=0.65,
                       float bad_val=0):
    
    cdef int scn_width = vi_diff_data.shape[0]
    cdef int scn_height = vi_diff_data.shape[1]
    
    cdef int x_0 = 0
    cdef int x_1 = scn_width
    cdef int y_0 = 0
    cdef int y_1 = scn_height
    
    # Counters for main loop
    cdef int x = 0
    cdef int y = 0
    cdef int wsize = min_win
    cdef int cur_win[4]
    
    # Intermediate variables
    cdef float sum_tot = 0
    cdef int n = 0
    cdef float cen_vid = 0
    cdef float cen_btd = 0
    cdef float cen_mir = 0
    cdef float cen_irr = 0
    cdef float cen_lwi = 0
    cdef float res[3] 
    cdef float btd_eqn[4]
    
    # BTD equation defines BTD threshold as a function of LW BT.
    # [0]: Max BTD threshold, for LWBT>[2]
    # [1]: Min BTD threshold, for LWBT<[3]
    # [2]: Max LW BT for threshold (K)
    # [3]: Min LW BT for threshold (K)
    btd_eqn[0] = 8
    btd_eqn[1] = 30
    btd_eqn[2] = 290
    btd_eqn[3] = 265
        
    # Output datasets
    cdef np.ndarray[dtype=np.float32_t, ndim=3] outarr = np.zeros((scn_width, scn_height, 4), dtype=np.single)
    cdef float[:,:, ::1] outarr_view = outarr
    
    # Create window array, set size to maximum usable, 15x15 pixels
    cdef np.ndarray[dtype=int, ndim=2] gpix = np.ones((15, 15), dtype=int)
    cdef int[:,::1] gpix_view = gpix
    
    # Loop across all pixels in the image
    for x in range(x_0, x_1):
        for y in range(y_0, y_1):
            # Ignore pixels that are not potential fire pixels
            if pfp[x, y] != 1:
                continue
            
            # Ignore fill value pixels
            cen_vid = vi_diff_data[x, y]
            if cen_vid == bad_val:
                continue
                
            # Loop over window sizes
            for wsize in range(min_win, max_win + 1):
                # Determine coordinates of the current window
                cur_win = get_curwindow_pos(x, y, wsize, scn_width, scn_height)
                # Don't process if any coords are bad (usually at extreme edges of image)
                if cur_win[0] < 0:
                    continue
                    
                cen_btd = btd_data[x, y]
                cen_mir = mir_bt_data[x, y]
                cen_irr = irr_data[x, y]
                    
                cur_btd = btd_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur_mir = mir_bt_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur_irr = irr_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                cur_lwi = lwi_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                
                # Compute mean and standard deviation across window
                cur_vid = vi_diff_data[cur_win[0]:cur_win[1], cur_win[2]:cur_win[3]]
                res = get_window_mea_stdv(wsize,
                                          cur_vid, cen_vid,
                                          cur_mir, cen_mir,
                                          cur_btd, cen_btd,
                                          cur_irr, cen_irr,
                                          cur_lwi,
                                          btd_eqn)
                
                # If we have good results, stop the loop.
                if res[1] > -100:
                    break
            # Set output array to:
            # [0]: Percentage of acceptable pixels in window
            # [1]: Mean window value
            # [2]: Standard deviation in the window
            # [3]: Window size that successfully processed
            outarr_view[x, y, 0] = res[0]
            outarr_view[x, y, 1] = res[1]
            outarr_view[x, y, 2] = res[2]
            outarr_view[x, y, 3] = wsize
        
    return outarr    

In [5]:
outa = get_mea_std_window(vi_diff_arr, btd_arr, mir_arr, irr_arr, lwi_arr, np.array(stg1_output), max_win=11)
perc_good = np.squeeze(outa[:,:,0])
mean_window = np.squeeze(outa[:,:,1])
std_window = np.squeeze(outa[:,:,2])
win_size = np.squeeze(outa[:,:,3])
#save_output(scn, perc_good, 'perc_good', odir, 'VI1_DIFF')
#save_output(scn, mean_window, 'mean', odir, 'VI1_DIFF')
#save_output(scn, std_window, 'std', odir, 'VI1_DIFF')
#save_output(scn, win_size, 'wsize', odir, 'VI1_DIFF')

In [6]:
new_arr = da.where(vi_diff_arr > mean_window + std_window, 1, 0)
new_arr = da.where(scn['PFP'] > 0, new_arr, 0)
new_arr = da.where(mean_window > -100, new_arr, 0)
save_output(scn, new_arr, 'stage2', odir, 'VI1_DIFF')

In [7]:
save_output(scn, perc_good, 'perc_good', odir, 'VI1_DIFF')
save_output(scn, mean_window, 'mean', odir, 'VI1_DIFF')
save_output(scn, std_window, 'std', odir, 'VI1_DIFF')
save_output(scn, win_size, 'wsize', odir, 'VI1_DIFF')

In [8]:
sw_15 = conv_kernel(scn['VI1_DIFF'], ksize=15)
rat_sw_15 = scn['VI1_DIFF'].data - sw_15

In [9]:
save_output(scn, sw_15 , 'sw_15', odir, 'VI1_DIFF')
save_output(scn, rat_sw_15 , 'rat_sw_15', odir, 'VI1_DIFF')

In [21]:
btd_win = np.array(conv_kernel(scn['BTD'], ksize=45))
btd = np.array(scn['BTD'])

btd_diff = btd - btd_win
btd_diff = np.where(btd_diff < 0, 1, btd_diff)

score_btd = np.power(btd_diff, 0.7) * np.power(btd - 5, 1.3) / 16

score_btd = np.where(lwi_arr < 270, 0, score_btd)

save_output(scn, score_btd , 'score_btd', odir, 'VI1_DIFF')

In [23]:
lwbt_win = np.array(conv_kernel(scn['LW1__BT'], ksize=45))
lwbt = np.array(scn['LW1__BT'])

lwbt_diff = lwbt - lwbt_win
lwbt_diff = np.where(lwbt_diff < 0, 1, lwbt_diff)

score_lwbt = np.power(lwbt_diff, 0.7) * np.power(lwbt - 270, 1.3) / 16
score_lwbt = np.where(lwi_arr < 270, 0, score_lwbt)
save_output(scn, score_lwbt, 'score_lwbt', odir, 'VI1_DIFF')

In [ ]:
lwbt_win = np.array(conv_kernel(scn['LW1__BT'], ksize=45))
lwbt = np.array(scn['LW1__BT'])

lwbt_diff = lwbt - lwbt_win
lwbt_diff = np.where(lwbt_diff < 0, 1, lwbt_diff)

score_lwbt = np.power(lwbt_diff, 0.7) * np.power(lwbt - 270, 1.3) / 16
score_lwbt = np.where(lwi_arr < 270, 0, score_lwbt)
save_output(scn, score_lwbt, 'score_lwbt', odir, 'VI1_DIFF')

In [13]:
out = np.zeros_like(score_lwbt)
out = np.where(score_btd > score_lwbt, 1, 0)
out = np.where(score_lwbt>0, out, 0)
save_output(scn, out, 'out', odir, 'VI1_DIFF')